# Finding putative metabolite interactions

We will employ [SMETANA]() through its [Python implementation]() to find putative metabolic interactions among the set of models in our use case: a phycosphere microbiome composed of the diatom: Pseudo-nitzschia sp., and the bacterial partners: Sulfitobacter sp., Alteromonas sp., Marinobacter sp., and Polaribacter sp.

To this end, we will simulate a marine medium containing only inorganic componets, as vitamins and cofactors will be synthesized by the bacterial partners, while the diatom will provide the carbon source.

## Which bacterial partners are producing Pseudo-nitzschia essential cofactors?

* Adenosylcobalamin (adocbl): _Sulfitobacter sp._
* Folate (fol): _?_
* Thiamin (thm): _?_ https://www.frontiersin.org/articles/10.3389/fmars.2020.606342/full


It seems like none of the reconstructed GEMs are producing thiamin and folate per se, but rather importing. Is this a limitation of the universal model or are these species really auxotrophs for them?

| Model | B12 (adocbl) | B1 (thm) | Folate |
|-------|----------|----------|--------|
| Sulfitobacter (00083)   | +   | -   | -   |
| Alteromonas (00080)   | -   | -   | -   |
| Marinobacter (00174)   | -   | -   | -   |
| Polaribacter (00201)   | -   | -   | -   |
| Pseudo-nitzschia (00212)   | -   | -   | -   |

In [3]:
%%bash

smetana \
    ../results/models/TARA_ARC_108*.xml \
    -m M9[marine] \
    --mediadb ../data/marine_media/media_db.tsv \
    -o ../results/bacterial_interactions \
    --solver gurobi \
    --detailed
    # --exclude ../data/compounds/inorganic.txt \ --molweight

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


In [4]:
import pandas as pd


df = pd.read_csv("../results/bacterial_interactions_detailed.tsv", sep="\t")
df

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
0,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00080.genepred,M_fe3pyovd_kt_e,0.5,0.39,1,0.195
1,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00080.genepred,M_nadp_e,0.5,0.80,1,0.400
2,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00080.genepred,M_thm_e,0.5,1.00,1,0.500
3,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00174.genepred,M_alaala_e,0.5,0.89,1,0.445
4,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00174.genepred,M_fe3pyovd_kt_e,0.5,0.39,1,0.195
5,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00174.genepred,M_h2_e,0.5,0.32,1,0.160
6,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00174.genepred,M_nadp_e,0.5,0.80,1,0.400
7,all,M9[marine],TARA_ARC_108_MAG_00117.genepred,TARA_ARC_108_MAG_00174.genepred,M_thm_e,0.5,1.00,1,0.500
8,all,M9[marine],TARA_ARC_108_MAG_00139.genepred,TARA_ARC_108_MAG_00080.genepred,M_fad_e,0.5,0.92,1,0.460
9,all,M9[marine],TARA_ARC_108_MAG_00139.genepred,TARA_ARC_108_MAG_00080.genepred,M_fe3pyovd_kt_e,0.5,0.02,1,0.010


## Which compounds can PS secrete?

In [38]:
import cobra 

models = {}
models["ps"] = cobra.io.read_sbml_model("../results/models/MAG_00212_pseudonitzschia_photoeuk_noSK.xml")
models["sulfito"] = cobra.io.read_sbml_model("../results/models/TARA_ARC_108_MAG_00083.genepred.xml")

In [34]:
from cobra import Model
from pathlib import Path


def get_medium_dict_from_media_db(media_db: Path, medium_id: str) -> dict:
    """
    Get a dictionary of exchange reactions for a given medium.

    Args:
        model (Model): _description_
        media_db (Path): _description_
        medium_id (str): _description_

    Returns:
        Model: _description_
    """
    media = pd.read_csv(media_db, sep="\t")
    if medium_id not in media.medium.values:
        raise ValueError(f"Medium {medium_id} not found in media database.")
    return {
    f"EX_{species}_e": 1000
    for species in media[media["medium"] == medium_id].compound
    }

def close_exchange_reactions(model: Model) -> Model:
    """
    Close all exchange reactions in a model.

    Args:
        model (Model): _description_

    Returns:
        Model: _description_
    """
    for rxn in model.reactions:
        if rxn.id.startswith("EX_"):
            rxn.lower_bound = 0
    return model

def set_medium(model: Model, medium_id: str, media_db: Path) -> Model:
    """
    Set the medium for a model.

    Args:
        model (Model): _description_
        medium_id (str): _description_
        media_db (Path): _description_

    Returns:
        Model: _description_
    """
    model = close_exchange_reactions(model)
    medium_dict = get_medium_dict_from_media_db(media_db, medium_id)
    medium_dict["EX_glc__D_e"] = -10
    for rxn_id, flux in medium_dict.items():
        if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).lower_bound = -flux
    return model

In [26]:
medium = get_medium_dict_from_media_db(
    "../data/marine_media/media_db.tsv", "M9[marine]"
    )
medium["EX_glc__D_e"] = 10
medium

{'EX_ca2_e': 1000,
 'EX_cl_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_fe3_e': 1000,
 'EX_glc__D_e': 10,
 'EX_h2o_e': 1000,
 'EX_h_e': 1000,
 'EX_k_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_na1_e': 1000,
 'EX_nh4_e': 1000,
 'EX_ni2_e': 1000,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000,
 'EX_so4_e': 1000,
 'EX_zn2_e': 1000,
 'EX_photon_e': 1000,
 'EX_co_e': 1000,
 'EX_co2_e': 1000,
 'EX_n2_e': 1000,
 'EX_hco3_e': 1000}

In [35]:
models["ps"] = set_medium(models["ps"], "M9[marine]", "../data/marine_media/media_db.tsv")

models["ps"].summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
photon_e,EX_photon_e,0.585,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


M9[marine]	M9 minimal medium (marine)	ca2	Ca2+
M9[marine]	M9 minimal medium (marine)	cl	Cl-
M9[marine]	M9 minimal medium (marine)	cobalt2	Co2+
M9[marine]	M9 minimal medium (marine)	cu2	Cu2+
M9[marine]	M9 minimal medium (marine)	fe2	Fe2+
M9[marine]	M9 minimal medium (marine)	fe3	Fe3+
M9[marine]	M9 minimal medium (marine)	glc__D	D-Glucose
M9[marine]	M9 minimal medium (marine)	h2o	H2O
M9[marine]	M9 minimal medium (marine)	h	H+
M9[marine]	M9 minimal medium (marine)	k	K+
M9[marine]	M9 minimal medium (marine)	mg2	Mg
M9[marine]	M9 minimal medium (marine)	mn2	Mn2+
M9[marine]	M9 minimal medium (marine)	mobd	Molybdate
M9[marine]	M9 minimal medium (marine)	na1	Na+
M9[marine]	M9 minimal medium (marine)	nh4	Ammonium
M9[marine]	M9 minimal medium (marine)	ni2	Ni2+
M9[marine]	M9 minimal medium (marine)	o2	O2
M9[marine]	M9 minimal medium (marine)	pi	Phosphate
M9[marine]	M9 minimal medium (marine)	so4	Sulfate
M9[marine]	M9 minimal medium (marine)	zn2	Zn2+
M9[marine]	M9 minimal medium (marine)	photon	photon (marine)
M9[marine]	M9 minimal medium (marine)	co	carbon monoxide
M9[marine]	M9 minimal medium (marine)	co2	carbon dioxyde
M9[marine]	M9 minimal medium (marine)	n2	Nitrogen
M9[marine]	M9 minimal medium (marine)	hco3	bicarbonate

In [4]:
models["ps"].reactions.get_by_id("EX_tsul_e")
models["ps"].metabolites.get_by_id("CPD-15199_c")

Metabolite identifier,CPD-15199_c
Name,5-deoxy-α-ribose 1-phosphate
Memory address,0x7fc95a69e190
Formula,C5H9O7P
Compartment,c
In 2 reaction(s),"DM_CPD-15199_c, RXN-14304"


In [16]:
from reconstruction import get_organic_exchanges
from cobra.flux_analysis.variability import flux_variability_analysis, find_blocked_reactions


carbon_exchanges = get_organic_exchanges(models["ps"])
fva_ex = flux_variability_analysis(models["ps"], reaction_list=carbon_exchanges, loopless=False, fraction_of_optimum=0.1)

In [18]:
fva_ex[fva_ex.maximum > 0]

,minimum,maximum
EX_glyclt_e,0.0,38.767932
EX_glc_e,0.0,18.674589
EX_tcynt_e,0.0,23.260759
EX_lac_d_e,0.0,10.000000
EX_etoh_e,0.0,74.698354
...,...,...
EX_ser_L_e,0.0,10.000000
EX_13dpg_e,0.0,10.000000
EX_2aobut_e,0.0,10.000000
EX_ala_B_e,0.0,10.000000


## What about exports in bacterial partners?

In [56]:
carbon_exchanges = get_organic_exchanges(models["sulfito"])

In [63]:
fva_ex = flux_variability_analysis(
    models["sulfito"],
    reaction_list=["CBLAT", "ADOCBLPP", "TR_adocbl_c_to_adocbl_e", "CBLATm", "EX_adocbl_e"],
    loopless=False,
    fraction_of_optimum=0.1
    )

fva_ex[fva_ex.maximum > 0]

fva_ex

,minimum,maximum
CBLAT,-1000.0,1000.000000
ADOCBLPP,0.0,1.077761
TR_adocbl_c_to_adocbl_e,0.0,0.000000
CBLATm,-1000.0,1000.000000
EX_adocbl_e,0.0,0.000000


In [73]:
models["sulfito"].reactions.get_by_id("ADOCBLPP")

Reaction identifier,ADOCBLPP
Name,Adenosylcobalamin phosphate phosphatase
Memory address,0x7fc9843d4b90
Stoichiometry,adocblp_c + h2o_p --> adocbl_c + pi_p Adenosylcobalamin-phosphate + H2O H2O --> Adenosylcobalamin + Phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [79]:
models["sulfito"].reactions.get_by_id("Growth").reaction.split(" + ")

['0.000223 10fthf_c',
 '0.513689 ala__L_c',
 '0.000223 amet_c',
 '0.295792 arg__L_c',
 '0.241055 asn__L_c',
 '0.241055 asp__L_c',
 '54.124831 atp_c',
 '0.005205 ca2_c',
 '0.005205 cl_c',
 '0.000576 coa_c',
 '0.0001 cobalt2_c',
 '0.133508 ctp_c',
 '0.000709 cu2_c',
 '0.09158 cys__L_c',
 '0.026166 datp_c',
 '0.027017 dctp_c',
 '0.027017 dgtp_c',
 '0.026166 dttp_c',
 '0.000223 fad_c',
 '0.006715 fe2_c',
 '0.007808 fe3_c',
 '0.26316 gln__L_c',
 '0.26316 glu__L_c',
 '0.612638 gly_c',
 '0.215096 gtp_c',
 '48.601527 h2o_c',
 '0.094738 his__L_c',
 '0.290529 ile__L_c',
 '0.195193 k_c',
 '0.450531 leu__L_c',
 '0.343161 lys__L_c',
 '0.153686 met__L_c',
 '0.008675 mg2_c',
 '0.000223 mlthf_c',
 '0.000691 mn2_c',
 '0.0001 mql8_c',
 '0.001831 nad_c',
 '0.000447 nadp_c',
 '0.185265 phe__L_c',
 '0.221055 pro__L_c',
 '0.000223 pydx5p_c',
 '0.000223 ribflv_c',
 '0.215792 ser__L_c',
 '0.004338 so4_c',
 '0.000223 thf_c',
 '0.000223 thmpp_c',
 '0.253687 thr__L_c',
 '0.056843 trp__L_c',
 '0.137896 tyr__L_c',

In [52]:
from future import __annotations__

from cobra import Model, Metabolite, Reaction


def add_external_metabolite(model: Model, met_id: str) -> Model:
    """
    Add an external metabolite to a model.

    Args:
        model (Model): _description_
        met_id (str): _description_

    Returns:
        Model: _description_
    """
    met = model.metabolites.get_by_id(met_id)
    met_to_add = met.copy()
    met_to_add.id = met_id[:-2] + "_e"
    met_to_add.compartment = "e"
    model.add_metabolites([met_to_add])
    return model

def add_transport_reaction(
        model: Model,
        met_i: str,
        met_j: str,
        lower_bound: float = -1000.0,
        upper_bound: float = 1000.0
        ) -> Model:
    """
    Add a transport reaction to a model.

    Args:
        model (Model): _description_
        met (Metabolite): _description_
        met_to_add (Metabolite): _description_

    Returns:
        Model: _description_
    """
    met = model.metabolites.get_by_id(met_i)
    met_to_add = model.metabolites.get_by_id(met_j)
    rxn_id = f"TR_{met.id}_to_{met_to_add.id}"
    rxn = Reaction(rxn_id)
    rxn.name = f"Transport of {met.id} to {met_to_add.id}"
    rxn.add_metabolites({met: -1, met_to_add: 1})
    rxn.lower_bound = lower_bound
    rxn.upper_bound = upper_bound
    rxn.subsystem = "Transport"
    rxn.gene_reaction_rule = "Spontaenous"
    model.add_reactions([rxn])
    return model

def add_exchanges_for_metabolites(model: Model, met_ids: list[str]) -> Model:
    """
    Add exchange reactions for a list of metabolites.

    Args:
        model (Model): _description_
        met_ids (list[str]): _description_

    Returns:
        Model: _description_
    """
    for met_id in met_ids:
        if met_id in model.metabolites:
            met = model.metabolites.get_by_id(met_id)
            if met.compartment != "e":
                model = add_external_metabolite(model, met_id)
                model = add_transport_reaction(model, met.id, met_to_add.id)
            else:
                met_to_add = met
            model.add_boundary(met_to_add, type="exchange")
        else:
            print(f"Metabolite {met_id} not found in model.")
    return model


models["sulfito"] = add_external_metabolite(models["sulfito"], "adocbl_c")
models["sulfito"] = add_transport_reaction(models["sulfito"], "adocbl_c", "adocbl_e")
models["sulfito"] = add_exchanges_for_metabolites(models["sulfito"], ["adocbl_e"])

In [70]:
models["sulfito"].metabolites.get_by_id("adocbl_e")


Metabolite identifier,adocbl_e
Name,Adenosylcobalamin
Memory address,0x7fc9842e13d0
Formula,C72H100CoN18O17P
Compartment,e
In 2 reaction(s),"EX_adocbl_e, TR_adocbl_c_to_adocbl_e"


In [72]:
models["sulfito"].reactions.get_by_id("TR_adocbl_c_to_adocbl_e")

Reaction identifier,TR_adocbl_c_to_adocbl_e
Name,Transport of adocbl_c to adocbl_e
Memory address,0x7fc9844673d0
Stoichiometry,adocbl_c <=> adocbl_e Adenosylcobalamin <=> Adenosylcobalamin
GPR,Spontaenous
Lower bound,-1000.0
Upper bound,1000.0


In [67]:
models["sulfito"].reactions.get_by_id("EX_cbl1_e")

KeyError: 'EX_cbl1_e'